In [33]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/pranay/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

In [34]:
flares = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


rindi = pd.read_csv('rindia.csv')
rindi.head()

,title,url,comms_num,flare,score,id,created,author,body,timestamp
0,Coronavirus (COVID-19) Megathread - News and U...,https://www.reddit.com/r/india/comments/fqqdsg...,8884,AskIndia,344,fqqdsg,1.585451e+09,IAmMohit,###[Covid-19 Fundraisers & Donation Links](htt...,2020-03-29 08:40:33
1,Coronavirus (Covid-19) Multi-Lingual Resources...,https://www.reddit.com/r/india/comments/fo0xj9...,49,AskIndia,214,fo0xj9,1.585063e+09,IAmMohit,This post will serve as a wiki for multi-lingu...,2020-03-24 20:53:48
2,The wealth inequality in India is truly horrif...,https://i.redd.it/3rol63nk35s41.jpg,671,AskIndia,8288,fyyx8c,1.586618e+09,sinsandtonic,NaN,2020-04-11 20:47:54
3,It's sad what passes for breaking news these days,https://i.imgur.com/F0UHNcw.jpg,37,AskIndia,519,fz6vdl,1.586646e+09,godzillaweener,NaN,2020-04-12 04:25:31
4,Railways transports 20 litres of camel milk to...,https://twitter.com/PTI_News/status/1248965677...,47,AskIndia,666,fz59yf,1.586641e+09,aviakki1,NaN,2020-04-12 03:00:18


In [35]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))



In [40]:
X = rindi.comms_n
y = rindi.flare

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [41]:
from sklearn.naive_bayes import MultinomialNB


nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flares))

AttributeError: 'int' object has no attribute 'lower'